# Predicting house prices

House sales in King County: https://www.kaggle.com/harlfoxem/housesalesprediction

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import scale

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

In [2]:

   
data=pd.read_csv("kc_house_data.csv")
year=[float(elem[:4]) for elem in data['date']]
year=np.array(year,dtype=np.float32)
year=year.reshape(-1,1)
    
month=[float(elem[4:6]) for elem in data['date']]
month=np.array(month,dtype=np.float32)
month=month.reshape(-1,1)
    
day=[float(elem[6:8]) for elem in data['date']]   
day=np.array(day,dtype=np.float32)
day=day.reshape(-1,1)
    
data=np.array(data)
data=np.append(data,year,axis=1)
data=np.append(data,month,axis=1)
data=np.append(data,day,axis=1)
X=data[:,[*range(3,24,1)]]
X=np.asarray(X).astype(np.float32)
Y=data[:,2]
Y=np.asarray(Y).astype(np.float32)
X_train_full, X_test, y_train_full, y_test = train_test_split(X,Y,test_size=0.2)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full,test_size=0.2)


# Trying first simple Linear Regression

In [3]:
from sklearn.linear_model import LinearRegression


In [5]:
lr=LinearRegression()

In [9]:
def get_results(model):
    history=model.fit(X_train_full, y_train_full)
    predictions_train=model.predict(X_train_full)
    predictions_test=model.predict(X_test)
    mse_train=mean_squared_error(predictions_train,y_train_full)
    mse_test=mean_squared_error(predictions_test,y_test)
    print('Train MSE:',mse_train,', Test MSE:',mse_test)

In [10]:
get_results(lr) #very bad results

Train MSE: 40431583000.0 , Test MSE: 39578567000.0


In [11]:
type(Y)

numpy.ndarray

# Trying support vector machine

In [3]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

In [13]:
types=['linear', 'poly', 'rbf', 'sigmoid']
params=dict(kernel=types)

In [14]:
support=SVR(kernel='sigmoid')

get_results(support)#even worse

Train MSE: 143964314388.81104 , Test MSE: 138580878686.50745


# ANNs

In [10]:
from tensorflow import keras

In [9]:
scaler = StandardScaler()
X_trains = scaler.fit_transform(X_train)
X_valids = scaler.transform(X_valid)
X_tests = scaler.transform(X_test)




In [8]:
X_trainn=normalize(X_train, norm='l2')
X_validn=normalize(X_valid, norm='l2')
X_testn=normalize(X_test, norm='l2')

In [79]:
input_=keras.layers.Input(shape=X_train.shape[1:])
hidden1=keras.layers.Dense(100,activation='relu')(input_)
hidden2=keras.layers.Dense(40,activation='relu')(hidden1)
output=keras.layers.Dense(1)(hidden2)
model=keras.Model(inputs=[input_],outputs=[output])

In [80]:
optimizer = keras.optimizers.Adam(clipvalue=0.5)

In [81]:
model.compile(loss="mean_squared_error", optimizer=optimizer)


In [83]:
history =model.fit(X_trainn, y_train, epochs=500,
                    validation_data=(X_validn, y_valid), callbacks=[EarlyStopping(patience=10)])


Epoch 1/500
433/433 [==============================] - 0s 915us/step - loss: 111758049280.0000 - val_loss: 130573910016.0000
Epoch 2/500
433/433 [==============================] - 0s 864us/step - loss: 111476146176.0000 - val_loss: 130273714176.0000
Epoch 3/500
433/433 [==============================] - 0s 878us/step - loss: 111286370304.0000 - val_loss: 130084118528.0000
Epoch 4/500
433/433 [==============================] - 0s 885us/step - loss: 111068545024.0000 - val_loss: 129742307328.0000
Epoch 5/500
433/433 [==============================] - 0s 878us/step - loss: 110773403648.0000 - val_loss: 129427701760.0000
Epoch 6/500
433/433 [==============================] - 0s 952us/step - loss: 110465974272.0000 - val_loss: 129155465216.0000
Epoch 7/500
433/433 [==============================] - 0s 1ms/step - loss: 110220271616.0000 - val_loss: 128862126080.0000
Epoch 8/500
433/433 [==============================] - 0s 1ms/step - loss: 109955432448.0000 - val_loss: 128509419520.0000
Epoc

Epoch 134/500
433/433 [==============================] - 0s 1ms/step - loss: 67949465600.0000 - val_loss: 77256720384.0000
Epoch 135/500
433/433 [==============================] - 0s 1ms/step - loss: 67926130688.0000 - val_loss: 77211189248.0000
Epoch 136/500
433/433 [==============================] - 1s 1ms/step - loss: 67931291648.0000 - val_loss: 77260627968.0000
Epoch 137/500
433/433 [==============================] - 1s 1ms/step - loss: 67942002688.0000 - val_loss: 77327556608.0000
Epoch 138/500
433/433 [==============================] - 1s 1ms/step - loss: 67857227776.0000 - val_loss: 77115244544.0000
Epoch 139/500
433/433 [==============================] - 0s 1ms/step - loss: 67813523456.0000 - val_loss: 77213589504.0000
Epoch 140/500
433/433 [==============================] - 0s 1ms/step - loss: 67744927744.0000 - val_loss: 77201203200.0000
Epoch 141/500
433/433 [==============================] - 0s 1ms/step - loss: 67755663360.0000 - val_loss: 76986556416.0000
Epoch 142/500
43

433/433 [==============================] - 0s 1ms/step - loss: 66021109760.0000 - val_loss: 75001511936.0000
Epoch 267/500
433/433 [==============================] - 0s 1ms/step - loss: 66035998720.0000 - val_loss: 75003748352.0000
Epoch 268/500
433/433 [==============================] - 0s 1ms/step - loss: 66016391168.0000 - val_loss: 75035025408.0000
Epoch 269/500
433/433 [==============================] - 0s 1ms/step - loss: 65984581632.0000 - val_loss: 74997563392.0000
Epoch 270/500
433/433 [==============================] - 0s 1ms/step - loss: 65965969408.0000 - val_loss: 74872250368.0000
Epoch 271/500
433/433 [==============================] - 0s 1ms/step - loss: 65950044160.0000 - val_loss: 74967121920.0000
Epoch 272/500
433/433 [==============================] - 0s 1ms/step - loss: 65926803456.0000 - val_loss: 74970996736.0000
Epoch 273/500
433/433 [==============================] - 0s 1ms/step - loss: 65942626304.0000 - val_loss: 74824785920.0000
Epoch 274/500
433/433 [=======

In [84]:
mse=model.evaluate(X_testn, y_test)

136/136 [==============================] - 0s 616us/step - loss: 65827020800.0000


In [85]:
pred_test=model.predict(X_testn)
mse_test=mean_squared_error(pred_test,y_test)

In [86]:
mse_test

65827040000.0

# Comparing to MLPRegressor

In [5]:
from sklearn.neural_network import MLPRegressor

In [6]:
nn= MLPRegressor(alpha=1e-5,hidden_layer_sizes=(100, 40), random_state=1)

In [11]:
nn.fit(X_trainn, y_train)

C:\Users\U6080396\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPRegressor(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100, 40), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=200,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=1, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [12]:
predi=nn.predict(X_testn)
mean_squared_error(predi,y_test)

124976988917.45087

# Tuning hyperparameters

In [27]:
def build_model(n_hidden=3, n_neurons=41, learning_rate=0.025989383726164616, input_shape=X_train.shape[1:]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=X_train.shape[1:]))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [14]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [17]:
keras_reg.fit(X_trainn, y_train, epochs=100,
              validation_data=(X_validn, y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])
mse_test = keras_reg.score(X_testn, y_test)
y_pred = keras_reg.predict(X_new)

Epoch 1/100
433/433 [==============================] - 1s 1ms/step - loss: 422410321920.0000 - val_loss: 439953555456.0000
Epoch 2/100
433/433 [==============================] - 0s 1ms/step - loss: 422046859264.0000 - val_loss: 439412817920.0000
Epoch 3/100
433/433 [==============================] - 0s 1ms/step - loss: 421358829568.0000 - val_loss: 438573432832.0000
Epoch 4/100
433/433 [==============================] - 0s 954us/step - loss: 420391518208.0000 - val_loss: 437470101504.0000
Epoch 5/100
433/433 [==============================] - 0s 926us/step - loss: 419177562112.0000 - val_loss: 436134215680.0000
Epoch 6/100
433/433 [==============================] - 0s 958us/step - loss: 417747664896.0000 - val_loss: 434592612352.0000
Epoch 7/100
433/433 [==============================] - 0s 931us/step - loss: 416121815040.0000 - val_loss: 432862068736.0000
Epoch 8/100
433/433 [==============================] - 0s 988us/step - loss: 414313742336.0000 - val_loss: 430953693184.0000
Epoch 

433/433 [==============================] - 1s 1ms/step - loss: 170424467456.0000 - val_loss: 184125145088.0000
Epoch 68/100
433/433 [==============================] - 1s 2ms/step - loss: 166918029312.0000 - val_loss: 180636647424.0000
Epoch 69/100
433/433 [==============================] - 1s 2ms/step - loss: 163541811200.0000 - val_loss: 177276600320.0000
Epoch 70/100
433/433 [==============================] - 1s 1ms/step - loss: 160310214656.0000 - val_loss: 174067679232.0000
Epoch 71/100
433/433 [==============================] - 1s 1ms/step - loss: 157210984448.0000 - val_loss: 170987094016.0000
Epoch 72/100
433/433 [==============================] - 1s 1ms/step - loss: 154254458880.0000 - val_loss: 168060485632.0000
Epoch 73/100
433/433 [==============================] - 1s 1ms/step - loss: 151439704064.0000 - val_loss: 165269143552.0000
Epoch 74/100
433/433 [==============================] - 1s 1ms/step - loss: 148775895040.0000 - val_loss: 162629697536.0000
Epoch 75/100
433/433 

NameError: name 'X_new' is not defined

In [18]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 110,10),
    "learning_rate": reciprocal(3e-4, 3e-2),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)
rnd_search_cv.fit(X_trainn, y_train, epochs=100,
                  validation_data=(X_validn, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100
289/289 [==============================] - 0s 2ms/step - loss: 412747956224.0000 - val_loss: 440007589888.0000
Epoch 2/100
289/289 [==============================] - 0s 1ms/step - loss: 412482371584.0000 - val_loss: 439603265536.0000
Epoch 3/100
289/289 [==============================] - ETA: 0s - loss: 411852275712.00 - 0s 1ms/step - loss: 411942977536.0000 - val_loss: 438903111680.0000
Epoch 4/100
289/289 [==============================] - 0s 2ms/step - loss: 411102740480.0000 - val_loss: 437923282944.0000
Epoch 5/100
289/289 [==============================] - 0s 1ms/step - loss: 410020020224.0000 - val_loss: 436718796800.0000
Epoch 6/100
289/289 [==============================] - 0s 1ms/step - loss: 408725716992.0000 - val_loss: 435313901568.0000
Epoch 7/100
289/289 [==============================] - 0s 2ms/step - loss: 407242604544.0000 - val_loss: 433722851328.0000
Epoch 8/100
289/289 [==============================] - 1s 2ms/step - loss: 405579825152.0000 - val_loss: 

Epoch 67/100
289/289 [==============================] - 0s 2ms/step - loss: 180243988480.0000 - val_loss: 202816913408.0000
Epoch 68/100
289/289 [==============================] - 0s 2ms/step - loss: 176615456768.0000 - val_loss: 199159365632.0000
Epoch 69/100
289/289 [==============================] - 0s 2ms/step - loss: 173068730368.0000 - val_loss: 195604480000.0000
Epoch 70/100
289/289 [==============================] - 0s 2ms/step - loss: 169600876544.0000 - val_loss: 192104873984.0000
Epoch 71/100
289/289 [==============================] - 0s 2ms/step - loss: 166238060544.0000 - val_loss: 188719480832.0000
Epoch 72/100
289/289 [==============================] - 1s 2ms/step - loss: 162971009024.0000 - val_loss: 185430114304.0000
Epoch 73/100
289/289 [==============================] - 1s 2ms/step - loss: 159815385088.0000 - val_loss: 182257893376.0000
Epoch 74/100
289/289 [==============================] - 1s 2ms/step - loss: 156766420992.0000 - val_loss: 179190382592.0000
Epoch 75

289/289 [==============================] - 0s 2ms/step - loss: 315620851712.0000 - val_loss: 330719002624.0000
Epoch 33/100
289/289 [==============================] - 0s 1ms/step - loss: 310506389504.0000 - val_loss: 325547982848.0000
Epoch 34/100
289/289 [==============================] - 0s 1ms/step - loss: 305367384064.0000 - val_loss: 320337575936.0000
Epoch 35/100
289/289 [==============================] - 0s 1ms/step - loss: 300182306816.0000 - val_loss: 315089649664.0000
Epoch 36/100
289/289 [==============================] - 0s 1ms/step - loss: 294967607296.0000 - val_loss: 309810495488.0000
Epoch 37/100
289/289 [==============================] - 0s 1ms/step - loss: 289731149824.0000 - val_loss: 304509255680.0000
Epoch 38/100
289/289 [==============================] - 0s 2ms/step - loss: 284494725120.0000 - val_loss: 299211227136.0000
Epoch 39/100
289/289 [==============================] - 0s 1ms/step - loss: 279231889408.0000 - val_loss: 293894848512.0000
Epoch 40/100
289/289 

Epoch 98/100
289/289 [==============================] - 0s 2ms/step - loss: 125736083456.0000 - val_loss: 140169347072.0000
Epoch 99/100
289/289 [==============================] - 0s 1ms/step - loss: 125705388032.0000 - val_loss: 140136316928.0000
Epoch 100/100
145/145 [==============================] - 0s 903us/step - loss: 125842202624.0000
Epoch 1/100
289/289 [==============================] - 1s 2ms/step - loss: 433054318592.0000 - val_loss: 439983407104.0000
Epoch 2/100
289/289 [==============================] - 1s 2ms/step - loss: 432729620480.0000 - val_loss: 439483662336.0000
Epoch 3/100
289/289 [==============================] - 1s 2ms/step - loss: 432073965568.0000 - val_loss: 438675111936.0000
Epoch 4/100
289/289 [==============================] - 0s 2ms/step - loss: 431128412160.0000 - val_loss: 437589573632.0000
Epoch 5/100
289/289 [==============================] - 0s 2ms/step - loss: 429921206272.0000 - val_loss: 436257095680.0000
Epoch 6/100
289/289 [===================

289/289 [==============================] - 0s 1ms/step - loss: 207593586688.0000 - val_loss: 212087455744.0000
Epoch 62/100
289/289 [==============================] - 0s 1ms/step - loss: 203503370240.0000 - val_loss: 208028139520.0000
Epoch 63/100
289/289 [==============================] - 0s 1ms/step - loss: 199492009984.0000 - val_loss: 204061851648.0000
Epoch 64/100
289/289 [==============================] - 0s 2ms/step - loss: 195567108096.0000 - val_loss: 200169259008.0000
Epoch 65/100
289/289 [==============================] - 1s 2ms/step - loss: 191748702208.0000 - val_loss: 196407066624.0000
Epoch 66/100
289/289 [==============================] - 0s 1ms/step - loss: 188023013376.0000 - val_loss: 192730365952.0000
Epoch 67/100
289/289 [==============================] - 0s 1ms/step - loss: 184396365824.0000 - val_loss: 189147807744.0000
Epoch 68/100
289/289 [==============================] - 0s 2ms/step - loss: 180883537920.0000 - val_loss: 185693683712.0000
Epoch 69/100
289/289 

...

289/289 [==============================] - 0s 1ms/step - loss: 412663939072.0000 - val_loss: 440011194368.0000
Epoch 69/100
289/289 [==============================] - 0s 1ms/step - loss: 412662136832.0000 - val_loss: 440009293824.0000
Epoch 70/100
289/289 [==============================] - 0s 1ms/step - loss: 412660236288.0000 - val_loss: 440007393280.0000
Epoch 71/100
289/289 [==============================] - 0s 1ms/step - loss: 412658532352.0000 - val_loss: 440005525504.0000
Epoch 72/100
289/289 [==============================] - 0s 1ms/step - loss: 412656533504.0000 - val_loss: 440003756032.0000
Epoch 73/100
289/289 [==============================] - 0s 1ms/step - loss: 412654698496.0000 - val_loss: 440001921024.0000
Epoch 74/100
289/289 [==============================] - 0s 1ms/step - loss: 412653060096.0000 - val_loss: 440000053248.0000
Epoch 75/100
289/289 [==============================] - 0s 1ms/step - loss: 412651290624.0000 - val_loss: 439998152704.0000
Epoch 76/100
289/289 

289/289 [==============================] - 0s 2ms/step - loss: 421458411520.0000 - val_loss: 440076337152.0000
Epoch 34/100
289/289 [==============================] - 0s 2ms/step - loss: 421456314368.0000 - val_loss: 440074338304.0000
Epoch 35/100
289/289 [==============================] - 0s 1ms/step - loss: 421454675968.0000 - val_loss: 440072536064.0000
Epoch 36/100
289/289 [==============================] - 0s 1ms/step - loss: 421452709888.0000 - val_loss: 440070733824.0000
Epoch 37/100
289/289 [==============================] - 1s 2ms/step - loss: 421450940416.0000 - val_loss: 440068833280.0000
Epoch 38/100
289/289 [==============================] - 0s 1ms/step - loss: 421449007104.0000 - val_loss: 440067063808.0000
Epoch 39/100
289/289 [==============================] - 0s 1ms/step - loss: 421447434240.0000 - val_loss: 440065196032.0000
Epoch 40/100
289/289 [==============================] - 0s 1ms/step - loss: 421445435392.0000 - val_loss: 440063262720.0000
Epoch 41/100
289/289 

289/289 [==============================] - 0s 1ms/step - loss: 421337366528.0000 - val_loss: 439954505728.0000
Epoch 100/100
145/145 [==============================] - 0s 840us/step - loss: 424190279680.0000
Epoch 1/100
289/289 [==============================] - 0s 2ms/step - loss: 433102225408.0000 - val_loss: 440135090176.0000
Epoch 2/100
289/289 [==============================] - 0s 1ms/step - loss: 433100128256.0000 - val_loss: 440133222400.0000
Epoch 3/100
289/289 [==============================] - 0s 1ms/step - loss: 433098457088.0000 - val_loss: 440131354624.0000
Epoch 4/100
289/289 [==============================] - 0s 1ms/step - loss: 433096425472.0000 - val_loss: 440129716224.0000
Epoch 5/100
289/289 [==============================] - 0s 1ms/step - loss: 433094656000.0000 - val_loss: 440127651840.0000
Epoch 6/100
289/289 [==============================] - 0s 1ms/step - loss: 433092853760.0000 - val_loss: 440125947904.0000
Epoch 7/100
289/289 [==============================] -

289/289 [==============================] - 0s 2ms/step - loss: 432986030080.0000 - val_loss: 440018894848.0000
Epoch 65/100
289/289 [==============================] - 0s 1ms/step - loss: 432984096768.0000 - val_loss: 440017027072.0000
Epoch 66/100
289/289 [==============================] - 0s 2ms/step - loss: 432982327296.0000 - val_loss: 440015126528.0000
Epoch 67/100
289/289 [==============================] - 0s 1ms/step - loss: 432980459520.0000 - val_loss: 440013291520.0000
Epoch 68/100
289/289 [==============================] - 0s 1ms/step - loss: 432978755584.0000 - val_loss: 440011423744.0000
Epoch 69/100
289/289 [==============================] - 0s 1ms/step - loss: 432976658432.0000 - val_loss: 440009687040.0000
Epoch 70/100
289/289 [==============================] - 0s 1ms/step - loss: 432974790656.0000 - val_loss: 440007589888.0000
Epoch 71/100
289/289 [==============================] - 0s 1ms/step - loss: 432973053952.0000 - val_loss: 440005885952.0000
Epoch 72/100
289/289 

RuntimeError: Cannot clone object <tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x000001FE703DD948>, as the constructor either does not set or modifies parameter learning_rate

In [19]:
rnd_search_cv.best_params_

{'learning_rate': 0.025989383726164616, 'n_hidden': 3, 'n_neurons': 41}

In [28]:

keras_reg_best = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [29]:
keras_reg_best.fit(X_trainn, y_train, epochs=500,
              validation_data=(X_validn, y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])
mse_test = keras_reg_best.score(X_testn, y_test)


Epoch 1/500
433/433 [==============================] - 1s 2ms/step - loss: 162439331840.0000 - val_loss: 131659759616.0000
Epoch 2/500
433/433 [==============================] - 1s 2ms/step - loss: 107341479936.0000 - val_loss: 97262501888.0000
Epoch 3/500
433/433 [==============================] - 1s 2ms/step - loss: 68862107648.0000 - val_loss: 66022395904.0000
Epoch 4/500
433/433 [==============================] - 1s 2ms/step - loss: 59630227456.0000 - val_loss: 58759254016.0000
Epoch 5/500
433/433 [==============================] - 1s 2ms/step - loss: 58171600896.0000 - val_loss: 56622772224.0000
Epoch 6/500
433/433 [==============================] - 1s 2ms/step - loss: 57783365632.0000 - val_loss: 59114938368.0000
Epoch 7/500
433/433 [==============================] - 1s 2ms/step - loss: 57036853248.0000 - val_loss: 54685040640.0000
Epoch 8/500
433/433 [==============================] - 1s 2ms/step - loss: 57345204224.0000 - val_loss: 52911263744.0000
Epoch 9/500
433/433 [========

In [32]:
final_pred=keras_reg_best.predict(X_testn)

In [33]:
mean_squared_error(final_pred, y_test)

57196003000.0